### Installing TPOT (Tree based Pipeline Optimization Technique)

In [1]:
# pip install deap update_checker tqdm
# pip install tpo

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [3]:
# preprocessing
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

train.shape , test.shape

((8523, 12), (5681, 11))

In [4]:
train.head(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [5]:
test.head(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [6]:
#Mean imputation
train['Item_Weight'].fillna(train['Item_Weight'].mean(),inplace=True)
test['Item_Weight'].fillna(test['Item_Weight'].mean(),inplace=True)

In [7]:
train['Item_Fat_Content'].replace(['low fat','LF'], ['Low Fat','Low Fat'],inplace=True)
test['Item_Fat_Content'].replace(['low fat','LF'], ['Low Fat','Low Fat'],inplace=True)

In [8]:
test['Item_Fat_Content'].replace('reg','Regular',inplace=True)
train['Item_Fat_Content'].replace('reg','Regular',inplace=True)

In [9]:
train['Outlet_Establishment_Year'] = 2013 - train['Outlet_Establishment_Year']
test['Outlet_Establishment_Year'] = 2013 - test['Outlet_Establishment_Year']

In [10]:
train['Outlet_Size'].fillna('Small',inplace=True)
test['Outlet_Size'].fillna('Small',inplace=True)

In [11]:
col = ['Outlet_Size','Outlet_Location_Type','Outlet_Type','Item_Fat_Content']
test['Item_Outlet_Sales'] = 0

In [12]:
combi = train.append(test)

In [13]:
combi.shape

(14204, 12)

In [14]:
number = LabelEncoder()

In [15]:
for i in col:
    combi[i] = number.fit_transform(combi[i].astype('str'))
    combi[i] = combi[i].astype('int')

In [16]:
train = combi[:train.shape[0]]
test = combi[train.shape[0]:]

In [17]:
train.shape, test.shape

((8523, 12), (5681, 12))

In [18]:
test.drop('Item_Outlet_Sales',axis=1,inplace=True)

In [19]:
tpot_train = train.drop(['Outlet_Identifier','Item_Type','Item_Identifier'],axis=1)
tpot_test = test.drop(['Outlet_Identifier','Item_Type','Item_Identifier'],axis=1)

In [20]:
target = tpot_train['Item_Outlet_Sales']
tpot_train.drop('Item_Outlet_Sales',axis=1,inplace=True)

## TPOT implementation

In [21]:
from tpot import TPOTRegressor

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tpot_train, target,
 train_size=0.75, test_size=0.25)

C:\Users\Abhijith\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
C:\Users\Abhijith\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\Abhijith\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [22]:
tpot = TPOTRegressor(generations=5,population_size=50,verbosity=2)
# generations is like epoch, population_size is the size of population for the breeding
# verbosity is the amount of data TPOT should show , for 2 it will show a status bar.
tpot.fit(X_train,y_train)
print(tpot.score(X_test,y_test))

# Export the optimal value into a python file.
tpot.export('tpot_boston_pipeline.py')

C:\Users\Abhijith\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


Generation 1 - Current best internal CV score: -1165008.7097997733


Generation 2 - Current best internal CV score: -1165008.7097997733


Generation 3 - Current best internal CV score: -1165008.7097997733


Generation 4 - Current best internal CV score: -1161583.618946705


Generation 5 - Current best internal CV score: -1157192.9959915741



Best pipeline: ExtraTreesRegressor(input_matrix, bootstrap=True, max_features=0.8, min_samples_leaf=15, min_samples_split=10, n_estimators=100)
-1175085.6887825355


True

We can see that the best Model for the problem is ExtraTreesRegressor 